## HW_3
### Практическое задание
  1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора)  на датасет from sklearn.datasets import load_boston. 
  2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу  Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

  3. Поработайте с документацией TensorFlow 2. Найти 2-3 полезные команды TensorFlow, не разобранные на уроке (полезные для Вас).

  1-2. (*)  Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете imdb_reviews .Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?
    
    
</ol>

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import r2_score

from tensorflow.keras.datasets import boston_housing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [2]:
print(tf.__version__)
msg = tf.constant('TensorFlow 2.0 Hello World')
tf.print(msg)

2.6.0
TensorFlow 2.0 Hello World


In [3]:
(X_train, y_train),(X_test, y_test)=boston_housing.load_data()

In [4]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [5]:
print(X_train.shape, X_test.shape)

(404, 13) (102, 13)


In [6]:
X_train

array([[-0.27224633, -0.48361547, -0.43576161, ...,  1.14850044,
         0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, ..., -1.71818909,
         0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , ...,  0.78447637,
         0.22061726, -1.30850006],
       ...,
       [-0.40202987,  0.99079651, -0.7415148 , ..., -0.71712291,
         0.07943894, -0.67776904],
       [-0.17292018, -0.48361547,  1.24588095, ..., -1.71818909,
        -0.98764362,  0.42083466],
       [-0.40422614,  2.04394792, -1.20161456, ..., -1.30866202,
         0.23317118, -1.15392266]])

Так как знаю не все лоссы и метрики, то просто добавил методом исключения:

In [7]:
losses = [
    tf.keras.losses.Huber,
    tf.keras.losses.KLDivergence,
    tf.keras.losses.LogCosh,
    tf.keras.losses.MeanAbsoluteError,
    tf.keras.losses.MeanAbsolutePercentageError,
    tf.keras.losses.MeanSquaredError,
    tf.keras.losses.MeanSquaredLogarithmicError,
]
metrics = [
    tf.keras.metrics.CosineSimilarity,
    tf.keras.metrics.Hinge,
    tf.keras.metrics.KLDivergence,
    tf.keras.metrics.LogCoshError,
    tf.keras.metrics.MeanAbsoluteError,
    tf.keras.metrics.MeanAbsolutePercentageError,
    tf.keras.metrics.MeanSquaredError,
    tf.keras.metrics.MeanSquaredLogarithmicError,
    tf.keras.metrics.Poisson,
    tf.keras.metrics.RootMeanSquaredError,
    tf.keras.metrics.SquaredHinge
]

In [8]:
models = []

In [9]:
callback = tf.keras.callbacks.EarlyStopping(
                              monitor="val_loss",
                              min_delta=0.0001,
                              patience=3,
                              verbose=1,
                              mode="auto",
                              baseline=None,
                              restore_best_weights=True,
                          )

сравнивать модели будем по знакомой и понятной метрике $R^2$

In [10]:
%%time

for current_loss in losses:
    for current_metric in metrics:
        name = f'{str(current_loss).split(".")[-1][:-2]}_{str(current_metric).split(".")[-1][:-2]}'
        print(name)
        model = Sequential([
            Dense(100, activation='relu', input_shape=(X_train.shape[1],)),
            Dense(50, activation='relu'),
            Dense(25, activation='relu'),
            Dense(10, activation='relu'),
            Dense(1)
        ])
        model.compile(
            optimizer='adam',
            loss=[current_loss()],
            metrics=[current_metric()]
        )
        history = model.fit(X_train, y_train,
                      epochs=10000,
                      batch_size=20, 
                      validation_split=0.2,
                      verbose=0,
                      callbacks=[callback],
                  )
        train_metrics = model.evaluate(X_train, y_train)
        test_metrics = model.evaluate(X_test, y_test)
        pred_train = model.predict(X_train)
        train_r2 = r2_score(y_train, pred_train)
        pred_test = model.predict(X_test)
        test_r2 = r2_score(y_test, pred_test)
        
        models.append({
            'name': name,
            'model': model,
            'history': history,
            'train_loss': train_metrics[0],
            'test_loss': test_metrics[0],
            'train_metric': train_metrics[1],
            'test_metric': test_metrics[1],
            'train_r2': train_r2,
            'test_r2': test_r2,
        })
        print()

Huber_CosineSimilarity
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 2.5886 - cosine_similarity: 1.0000

Huber_Hinge
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 2.4897 - hinge: 0.0000e+00

Huber_KLDivergence
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 2.5261 - kullback_leibler_divergence: 0.0000e+00

Huber_LogCoshError
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
4/4 [==============================] - 0s 3ms/step - loss: 2.5319 - logcosh: 2.4016

Huber_MeanAbsoluteError
Restoring model weights from the end of the best epoch.
Epoch 00033: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 2.4148 - mean_absolute_error: 2.8515

Huber_Mean

4/4 [==============================] - 0s 5ms/step - loss: 2.2914 - mean_absolute_error: 2.8697

LogCosh_MeanAbsolutePercentageError
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 2.3287 - mean_absolute_percentage_error: 15.1403

LogCosh_MeanSquaredError
Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
4/4 [==============================] - 0s 5ms/step - loss: 2.2361 - mean_squared_error: 16.5425

LogCosh_MeanSquaredLogarithmicError
Restoring model weights from the end of the best epoch.
Epoch 01514: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 6.0665 - mean_squared_logarithmic_error: 0.1672

LogCosh_Poisson
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 2.2507 - poisson: -50.7379

LogCosh_RootMeanSquaredError
Restoring model weights 

4/4 [==============================] - 0s 4ms/step - loss: 14.4320 - poisson: -50.7528

MeanAbsolutePercentageError_RootMeanSquaredError
Restoring model weights from the end of the best epoch.
Epoch 00032: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 14.2337 - root_mean_squared_error: 4.0475

MeanAbsolutePercentageError_SquaredHinge
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 14.7086 - squared_hinge: 0.0000e+00

MeanSquaredError_CosineSimilarity
Restoring model weights from the end of the best epoch.
Epoch 00032: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 24.0934 - cosine_similarity: 1.0000

MeanSquaredError_Hinge
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 25.6008 - hinge: 0.0000e+00

MeanSquaredError_KLDivergence
Restoring model w

In [11]:
results = pd.DataFrame(models).drop(['model', 'history'],axis=1)
results

,name,train_loss,test_loss,train_metric,test_metric,train_r2,test_r2
0,Huber_CosineSimilarity,1.822563,2.588642,1.000000,1.000000,0.850980,0.752793
1,Huber_Hinge,1.788935,2.489674,0.000000,0.000000,0.847274,0.781944
2,Huber_KLDivergence,1.873441,2.526053,0.000000,0.000000,0.837377,0.762149
3,Huber_LogCoshError,1.790627,2.531869,1.682698,2.401638,0.846779,0.771626
4,Huber_MeanAbsoluteError,1.679937,2.414753,2.106008,2.851494,0.866358,0.779942
...,...,...,...,...,...,...,...
72,MeanSquaredLogarithmicError_MeanSquaredError,9.634681,9.806747,604.020203,635.596130,-6.137841,-6.635353
73,MeanSquaredLogarithmicError_MeanSquaredLogarit...,9.634681,9.806747,9.634681,9.806747,-6.027299,-6.510259
74,MeanSquaredLogarithmicError_Poisson,0.021758,0.042951,-48.718491,-50.704308,0.839641,0.762187
75,MeanSquaredLogarithmicError_RootMeanSquaredError,0.021062,0.045442,3.549192,4.683145,0.851141,0.736535


In [12]:
results.sort_values(by=['test_r2'], ascending=False)[:20]

,name,train_loss,test_loss,train_metric,test_metric,train_r2,test_r2
48,MeanAbsolutePercentageError_MeanAbsoluteError,10.301604,14.314724,2.266057,2.812323,0.818855,0.812048
53,MeanAbsolutePercentageError_RootMeanSquaredError,10.632647,14.233661,3.726139,4.047540,0.835928,0.803198
28,LogCosh_MeanSquaredError,1.625854,2.236084,12.221090,16.542501,0.855581,0.801277
7,Huber_MeanSquaredLogarithmicError,1.902591,2.431320,0.023448,0.040783,0.821507,0.800495
34,MeanAbsoluteError_Hinge,2.141349,2.789681,0.000000,0.000000,0.837433,0.799163
52,MeanAbsolutePercentageError_Poisson,10.079053,14.431956,-48.682350,-50.752823,0.817363,0.797132
27,LogCosh_MeanAbsolutePercentageError,1.731798,2.328740,10.765066,15.140324,0.834882,0.791863
31,LogCosh_RootMeanSquaredError,1.628946,2.250333,3.492333,4.168153,0.855873,0.791294
50,MeanAbsolutePercentageError_MeanSquaredError,10.322030,13.745417,15.876963,17.436861,0.812378,0.790533
47,MeanAbsolutePercentageError_LogCoshError,9.689545,13.907456,1.612969,2.189927,0.829799,0.788785


Похоже, что что-то они могут)

Выберем первую строку и попытемся подобрать слои и нейроны

In [13]:
loss = tf.keras.losses.MeanAbsolutePercentageError
metric = tf.keras.metrics.MeanAbsoluteError

In [14]:
start_neurons = [50, 100, 200, 500]
layers = list(range(1, 6))
models_2 = []

Немного уточним дельту

In [15]:
callback = tf.keras.callbacks.EarlyStopping(
                              monitor="val_loss",
                              min_delta=0.00001,
                              patience=3,
                              verbose=1,
                              mode="auto",
                              baseline=None,
                              restore_best_weights=True,
                          )

In [16]:
%%time 

for layer in layers:
    for start in start_neurons:
        schema = [start]
        for i in range(layer):
            schema.append(int(0.7 * schema[-1]))
        name = str(schema)
        
        model = Sequential()
        model.add(Dense(start, activation='relu', input_shape=(X_train.shape[1],)))
        for i in schema[1:]:
            model.add(Dense(i, activation='relu'))
        model.add(Dense(1))
        
        model.compile(
                optimizer='adam',
                loss=[loss()],
                metrics=[metric()]
            )
        history = model.fit(X_train, y_train,
                      epochs=10000,
                      batch_size=20, 
                      validation_split=0.2,
                      verbose=0,
                      callbacks=[callback],
                  )
        train_metrics = model.evaluate(X_train, y_train)
        test_metrics = model.evaluate(X_test, y_test)
        pred_train = model.predict(X_train)
        train_r2 = r2_score(y_train, pred_train)
        pred_test = model.predict(X_test)
        test_r2 = r2_score(y_test, pred_test)

        models_2.append({
            'name': name,
            'model': model,
            'history': history,
            'train_loss': train_metrics[0],
            'test_loss': test_metrics[0],
            'train_metric': train_metrics[1],
            'test_metric': test_metrics[1],
            'train_r2': train_r2,
            'test_r2': test_r2,
        })
        print()

Restoring model weights from the end of the best epoch.
Epoch 00051: early stopping
4/4 [==============================] - 0s 3ms/step - loss: 14.1252 - mean_absolute_error: 2.8417

Restoring model weights from the end of the best epoch.
Epoch 00029: early stopping
4/4 [==============================] - 0s 3ms/step - loss: 14.2167 - mean_absolute_error: 2.9360

Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
4/4 [==============================] - 0s 4ms/step - loss: 15.4370 - mean_absolute_error: 3.1364

Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
4/4 [==============================] - 0s 3ms/step - loss: 15.0094 - mean_absolute_error: 3.0110

Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
4/4 [==============================] - 0s 5ms/step - loss: 15.4781 - mean_absolute_error: 3.1081

Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
4/4 [=

In [19]:
results_2 = pd.DataFrame(models_2).drop(['model', 'history'],axis=1)
results_2

,name,train_loss,test_loss,train_metric,test_metric,train_r2,test_r2
0,"[50, 35]",9.565228,14.125244,2.191763,2.841683,0.807875,0.808256
1,"[100, 70]",10.158446,14.216733,2.341311,2.936005,0.774937,0.791986
2,"[200, 140]",10.926607,15.437003,2.452211,3.136356,0.800460,0.766523
3,"[500, 350]",10.686156,15.009403,2.326963,3.011032,0.825322,0.784842
4,"[50, 35, 24]",10.836260,15.478106,2.383885,3.108092,0.811500,0.771000
5,"[100, 70, 49]",9.990983,14.764905,2.348269,3.151393,0.787439,0.765578
6,"[200, 140, 98]",10.991488,14.818805,2.465712,3.095351,0.798755,0.777826
7,"[500, 350, 244]",10.901162,15.450741,2.416084,3.089428,0.804855,0.769593
8,"[50, 35, 24, 16]",10.492477,13.912704,2.321939,2.800629,0.811583,0.796630
9,"[100, 70, 49, 34]",9.952058,14.412814,2.255174,3.048877,0.817775,0.772127


In [20]:
results_2.sort_values(by=['test_r2'], ascending=False)[:20]

,name,train_loss,test_loss,train_metric,test_metric,train_r2,test_r2
12,"[50, 35, 24, 16, 11]",10.437220,14.729230,2.292365,2.920325,0.815367,0.814198
0,"[50, 35]",9.565228,14.125244,2.191763,2.841683,0.807875,0.808256
8,"[50, 35, 24, 16]",10.492477,13.912704,2.321939,2.800629,0.811583,0.796630
1,"[100, 70]",10.158446,14.216733,2.341311,2.936005,0.774937,0.791986
3,"[500, 350]",10.686156,15.009403,2.326963,3.011032,0.825322,0.784842
18,"[200, 140, 98, 68, 47, 32]",10.867265,14.951520,2.371538,3.010934,0.819357,0.782751
6,"[200, 140, 98]",10.991488,14.818805,2.465712,3.095351,0.798755,0.777826
16,"[50, 35, 24, 16, 11, 7]",10.954453,15.217505,2.480672,3.103962,0.779832,0.774624
9,"[100, 70, 49, 34]",9.952058,14.412814,2.255174,3.048877,0.817775,0.772127
4,"[50, 35, 24]",10.836260,15.478106,2.383885,3.108092,0.811500,0.771000


Похоже, что такого подбора параметров недостаточно, так как топ слишком разнообразен, чтобы сделать вывод.